In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [2]:
url = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df = pd.read_csv(url)
df = df.sort_index(ascending=True, axis=0)
df = df.reset_index(drop=True)

# Splitting the dataset into training and testing
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))


In [3]:
# Define the window size for time series data
window_size = 60

# Prepare the training data
train_X, train_y = [], []
for i in range(window_size, len(train_data)):
    train_X.append(scaled_data[i-window_size:i, 0])
    train_y.append(scaled_data[i, 0])
train_X, train_y = np.array(train_X), np.array(train_y)

# Reshape the data for LSTM input [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))


In [4]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_X, train_y, epochs=10, batch_size=16)


Epoch 1/10
98/98 [==============================] - 9s 34ms/step - loss: 0.0043
Epoch 2/10
98/98 [==============================] - 4s 36ms/step - loss: 0.0010
Epoch 3/10
98/98 [==============================] - 4s 37ms/step - loss: 8.5594e-04
Epoch 4/10
98/98 [==============================] - 3s 34ms/step - loss: 7.6841e-04
Epoch 5/10
98/98 [==============================] - 3s 34ms/step - loss: 7.8455e-04
Epoch 6/10
98/98 [==============================] - 3s 34ms/step - loss: 7.1014e-04
Epoch 7/10
98/98 [==============================] - 4s 38ms/step - loss: 6.4820e-04
Epoch 8/10
98/98 [==============================] - 4s 38ms/step - loss: 5.7646e-04
Epoch 9/10
98/98 [==============================] - 4s 37ms/step - loss: 5.4122e-04
Epoch 10/10
98/98 [==============================] - 4s 37ms/step - loss: 5.0952e-04


In [5]:
# Prepare the test data
test_X, test_y = [], []
for i in range(window_size, len(test_data)):
    test_X.append(scaled_data[i-window_size:i, 0])
    test_y.append(scaled_data[i, 0])
test_X, test_y = np.array(test_X), np.array(test_y)

# Reshape the data for LSTM input [samples, time steps, features]
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

# Make predictions
predictions = model.predict(test_X)
predictions = scaler.inverse_transform(predictions)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - test_y) ** 2)))
print("RMSE:", rmse)


11/11 [==============================] - 2s 25ms/step
RMSE: 230.99110413917646
